In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install transformers[sentencepiece]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!pip install Xformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
!pip install transformers[torch]
!pip install accelerate -U

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [44]:
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 2.6 MB/s eta 0:00:00


In [6]:
# Load dataset
from datasets import load_dataset

imdb = load_dataset('imdb')

  0%|          | 0/3 [00:00<?, ?it/s]

In [7]:
from transformers import DistilBertTokenizer, DistilBertModel

checkpoint = "distilbert-base-uncased"

# Load pretrained tokenizer
tokenizer = DistilBertTokenizer.from_pretrained(checkpoint)

In [8]:
def tokenize_function(example):
  return tokenizer(example['text'], truncation=True)


tokenized_imdb = imdb.map(tokenize_function, batched=True)

In [9]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [10]:
from transformers import TrainingArguments

training_args = TrainingArguments("test-trainer")

In [11]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.weight', 'pre_classifier.bias', 'classifier.

In [35]:
train_dataset = tokenized_imdb["train"].train_test_split(
   stratify_by_column='label', seed=123456789, test_size=0.33 
)

In [37]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=train_dataset['train'],
    eval_dataset=train_dataset['test'],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [38]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
500,0.393400
1000,0.335700
1500,0.315300
2000,0.283900
2500,0.191200
3000,0.168600
3500,0.179000
4000,0.187400
4500,0.118200
5000,0.080100


TrainOutput(global_step=6282, training_loss=0.19223074562342368, metrics={'train_runtime': 2297.8884, 'train_samples_per_second': 21.868, 'train_steps_per_second': 2.734, 'total_flos': 6269750281890912.0, 'train_loss': 0.19223074562342368, 'epoch': 3.0})

In [41]:
predictions = trainer.predict(tokenized_imdb['test'])
print(predictions.predictions.shape, predictions.label_ids.shape)

(25000, 2) (25000,)


In [42]:
import numpy as np

preds = np.argmax(predictions.predictions, axis=-1)

In [49]:
from datasets import load_metric

metric = load_metric("accuracy")
metric.compute(predictions=preds, references=predictions.label_ids)

{'accuracy': 0.92416}

In [51]:
predictions

PredictionOutput(predictions=array([[ 3.5955832, -4.1300282],
       [ 3.1091669, -3.5490632],
       [ 3.6239245, -4.16572  ],
       ...,
       [-2.9756825,  2.4530537],
       [ 1.7860731, -2.0566251],
       [-4.078253 ,  3.3085337]], dtype=float32), label_ids=array([0, 0, 0, ..., 1, 1, 1]), metrics={'test_loss': 0.39542660117149353, 'test_runtime': 369.7187, 'test_samples_per_second': 67.619, 'test_steps_per_second': 8.452})

In [52]:
tokenized_imdb['test']

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 25000
})

In [64]:
def find_wrong_prediction(number: int):
  wrong_list = []
  for i in range(len(preds)):
    if preds[i] != tokenized_imdb['test']['label'][i]:
      wrong_list.append(i)
      if len(wrong_list) >= number:
        break
  return wrong_list

In [63]:
wrong_list = find_wrong_prediction(2)
for i in wrong_list:
  print(f"Prediction: {preds[i]} Expected: {tokenized_imdb['test']['label'][i]}")
  print(tokenized_imdb['test']['text'][i])

Prediction: 1 Expected: 0
First off let me say, If you haven't enjoyed a Van Damme movie since bloodsport, you probably will not like this movie. Most of these movies may not have the best plots or best actors but I enjoy these kinds of movies for what they are. This movie is much better than any of the movies the other action guys (Segal and Dolph) have thought about putting out the past few years. Van Damme is good in the movie, the movie is only worth watching to Van Damme fans. It is not as good as Wake of Death (which i highly recommend to anyone of likes Van Damme) or In hell but, in my opinion it's worth watching. It has the same type of feel to it as Nowhere to Run. Good fun stuff!
Prediction: 1 Expected: 0
Low budget horror movie. If you don't raise your expectations too high, you'll probably enjoy this little flick. Beginning and end are pretty good, middle drags at times and seems to go nowhere for long periods as we watch the goings on of the insane that add atmosphere but 

4. For at least 2 samples which have been wrongly classified in the test set, try explaining why the model could have been wrong.

**First sample:** "First off let me say, If you haven't enjoyed a Van Damme movie since bloodsport, you probably will not like this movie. Most of these movies may not have the best plots or best actors but I enjoy these kinds of movies for what they are. This movie is much better than any of the movies the other action guys (Segal and Dolph) have thought about putting out the past few years. Van Damme is good in the movie, the movie is only worth watching to Van Damme fans. It is not as good as Wake of Death (which i highly recommend to anyone of likes Van Damme) or In hell but, in my opinion it's worth watching. It has the same type of feel to it as Nowhere to Run. Good fun stuff!
"

In this comment, the used of double negation is important. Plus, the opinion part is small comparing to the rest of the comment.

**Second sample:** "Low budget horror movie. If you don't raise your expectations too high, you'll probably enjoy this little flick. Beginning and end are pretty good, middle drags at times and seems to go nowhere for long periods as we watch the goings on of the insane that add atmosphere but do not advance the plot. Quite a bit of gore. I enjoyed Bill McGhee's performance which he made quite believable for such a low budget picture, he managed to carry the movie at times when nothing much seemed to be happening. Nurse Charlotte Beale, played by Jesse Lee, played her character well so be prepared to want to slap her toward the end! She makes some really stupid mistakes but then, that's what makes these low budget movies so good! I would have been out of that place and five states away long before she even considered that it might be a good idea to leave! If you enjoy this movie, try Committed from 1988 which is basically a rip off of this movie."

In this comment, the author give the good parts of the movie.


5. What are the advantages and inconvenient of using this model in production compared to the naive Bayes we implemented in the first part of the course? And compared to a recurrent model like an RNN or an LSTM?

The naive Bayes only rely on the relation between words in a statistical way. This model can understand better relation between words and do not only rely on statistics.

The advantage of this model compared to a recurrent model is the capacity of capturing dependencies between words and sequential information.

